Unsupervised learning in trading involves using ML techniques to analyse data and discover patterns, relationships, and structures within the data without any pre-defined label or target variable. 

Unsupervised learning has multiple applications to trading
- clustering 
- dimensionality reduction 
- anomaly detection
- market regime detection 
- portfolio optimization 

Our plan for this project:
 - Download SP500 stocks prices data
 - Calculate different technical indicators and features for each stock 
 - Aggregate monthly and filter only top 150 most liquid stocks for each month
 - Calculate monthly returns for different time horizons 
 - Use Fama-French Factors to calculate rolling betas for each stock 
 - Train a K-Means model for each month to cluster similar stocks together
 - For each month, select assets based on a cluster and form a portfolio using Efficient Frontier max sharpe ratio portfolio optimisation 
 - Visualise portfolio returns and compare with SP500 

=> LIMITATION: We are using only the most recent SP500 stocks list and therefore suffer from the surviviorship bias. We should use surviviorship free data. 

### 1. Downloading Data

In [28]:
from statsmodels.regression.rolling import RollingOLS
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf
import pandas_ta
import warnings
warnings.filterwarnings("ignore")

sp500 = pd.read_html("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies")[0]

sp500["Symbol"] = sp500["Symbol"].replace(".", "-")
symbols_list = sp500["Symbol"].unique().tolist()

end_date = "2024-07-09"
start_date = pd.to_datetime(end_date)-pd.DateOffset(365*8)

df = yf.download(tickers = symbols_list, start=start_date, end=end_date).stack()

df.index.names = ['date','ticker']
df.columns = df.columns.str.lower()

df


[**                     5%%                      ]  27 of 502 completedFailed to get ticker 'BRK.B' reason: HTTPSConnectionPool(host='guce.yahoo.com', port=443): Read timed out. (read timeout=30)
[******                12%%                      ]  59 of 502 completed

$BF.B: possibly delisted; No price data found  (1d 2016-07-11 00:00:00 -> 2024-07-09)


[*********************100%%**********************]  501 of 502 completed

27 Failed downloads:
['ALB', 'IRM', 'SRE', 'NEM', 'BRO', 'ISRG', 'CTLT', 'VTR', 'JCI', 'CRWD', 'HPE']: ConnectionError(ProtocolError('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')))
[*********************100%%**********************]  501 of 502 completed['CSGP', 'GL', 'DECK', 'CLX', 'VRSN', 'LNT', 'CDW', 'PFE', 'UNP', 'PPL', 'V', 'SCHW', 'CI', 'REG']: ConnectionError(ReadTimeoutError("HTTPSConnectionPool(host='query2.finance.yahoo.com', port=443): Read timed out."))
['BRK.B']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')
['BF.B']: YFPricesMissingError('$%ticker%: possibly delisted; No price data found  (1d 2016-07-11 00:00:00 -> 2024-07-09)')


Price               adj close       close  ...        open      volume
date       ticker                          ...                        
2016-07-11 A        42.635681   45.400002  ...   45.610001   1094700.0
           AAL      29.945486   31.160000  ...   30.230000  12374400.0
           AAPL     22.268711   24.245001  ...   24.187500  95179600.0
           ABBV     45.603844   64.349998  ...   64.250000   9641500.0
           ABT      36.353951   42.119999  ...   42.029999   9052300.0
...                       ...         ...  ...         ...         ...
2024-07-08 XYL     134.059998  134.059998  ...  134.809998    746800.0
           YUM     127.940002  127.940002  ...  129.869995   1846100.0
           ZBH     106.389999  106.389999  ...  107.839996   1651100.0
           ZBRA    314.480011  314.480011  ...  311.989990    209100.0
           ZTS     175.429993  175.429993  ...  175.410004   1704700.0

[936346 rows x 6 columns]

### 2. Calculating features and techical indicators for each stock

For each stock, we will compute the following metrics:
 - Garman-Klass volatility
 - RSI
 - Bollinger Bands
 - ATR
 - MACD
 - Dollar volume

In [38]:
df["garman-klass volatility"] = ((np.log(df["high"]) - np.log(df["low"]))**2)/2 - (2*np.log(2)-1)*((np.log(df["adj close"])-np.log(df["open"]))**2)
df["rsi"] = df.groupby(level=1)["adj close"].transform(lambda x: pandas_ta.rsi(close=x,length=20))

# We want to normalise the indicators from this point. We normalise for better clustering.
df["bb_low"] = df.groupby(level=1)["adj close"].transform(lambda x: pandas_ta.bbands(close= np.log1p(x), length = 20).iloc[:,0])
df["bb_mid"] = df.groupby(level=1)["adj close"].transform(lambda x: pandas_ta.bbands(close= np.log1p(x), length = 20).iloc[:,1])
df["bb_high"] = df.groupby(level=1)["adj close"].transform(lambda x: pandas_ta.bbands(close= np.log1p(x), length = 20).iloc[:,2])

def compute_atr(stock_data):
    atr = pandas_ta.atr(high = stock_data["high"], low = stock_data["low"], close = stock_data["close"], length = 14)
    return atr.sub(atr.mean()).div(atr.std())

df["atr"] = df.groupby(level=1, group_keys=False).apply(compute_atr)

def compute_macd(close):
    macd = pandas_ta.macd(close = close, length = 20).iloc[:,0]
    return macd.sub(macd.mean()).div(macd.std())

df["macd"] = df.groupby(level=1, group_keys=False)["adj close"].apply(compute_macd)

df["dollar_volume"] = (df["close"]*df["volume"])/1e6

df

Price               adj close       close  ...      macd  dollar_volume
date       ticker                          ...                         
2016-07-11 A        42.635681   45.400002  ...       NaN      49.699382
           AAL      29.945486   31.160000  ...       NaN     385.586302
           AAPL     22.268711   24.245001  ...       NaN    2307.629482
           ABBV     45.603844   64.349998  ...       NaN     620.430510
           ABT      36.353951   42.119999  ...       NaN     381.282866
...                       ...         ...  ...       ...            ...
2024-07-08 XYL     134.059998  134.059998  ... -1.070050     100.116006
           YUM     127.940002  127.940002  ... -1.846067     236.190039
           ZBH     106.389999  106.389999  ... -0.868471     175.660528
           ZBRA    314.480011  314.480011  ... -0.004728      65.757770
           ZTS     175.429993  175.429993  ...  0.272868     299.055509

[936346 rows x 14 columns]

### 3. Aggregate on a monthly level and select top 150 most liquid stocks 

We do this aggregation to reduce treining time and therefore facilitate the experimentation with different features and strategies.

In [56]:
# Create a table containing only aggregates of those metrics we calculated + those that can be aggregated. 

last_cols = [c for c in df.columns.unique(0) if c not in ["dollar_volume","volume","open","high","low","close"]]

# dollar_volume is aggregated based on its mean over a month. All others use the value at a month's end 
# as the value for this month.
data = pd.concat([df.unstack('ticker')['dollar_volume'].resample('M').mean().stack("ticker").to_frame("dollar_volume"), 
df.unstack()[last_cols].resample("M").last().stack("ticker")],axis=1).dropna()

# Calculating dollar_volume 5-year rolling average for each stock before selecting
# data["dollar_volume"] = (data["dollar_volume"].unstack("ticker").rolling(5*12).mean().stack())

data["dollar_volume_rank"] = data.groupby("date")["dollar_volume"].rank(ascending=False)

# Selecting top 150 stocks in terms of dollar volume.
data = data[data["dollar_volume_rank"]<150].drop(["dollar_volume","dollar_volume_rank"],axis=1)

dollar_volume   adj close  ...      macd  dollar_volume_rank
date       ticker                             ...                              
2016-08-31 A           93.345724   44.119469  ... -0.131216               284.0
           AAL        279.896742   34.989563  ...  0.563338                82.0
           AAPL      2946.852230   24.494833  ... -0.117699                 1.0
           ABBV       432.525246   45.825455  ... -0.278459                42.0
           ABT        434.490414   36.491043  ... -0.311170                40.0
...                          ...         ...  ...       ...                 ...
2024-07-31 GEHC       152.356552   76.349998  ... -0.621724               286.0
           KVUE       253.954599   18.250000  ... -0.177861               171.0
           VLTO       131.537862   95.900002  ... -1.811718               324.0
           GEV        335.977917  174.369995  ... -1.359101               122.0
           SOLV        68.993884   49.639999  ... -0.899849               442.0

[44699 rows x 10 columns]

### 5. Calculate monthly returns over different time horizons and add them as features

We do this in order to reflect different time series dynamics which reflect the momentum patterns for each stock. We will compute over 1 month, 2 months, 3 months, 6 months, 9 months, and 12 months

In [59]:
def calculate_returns(df):

    lags = [1,2,3,6,9,12]

    # There will certainly be outliers in our data. If it's the case we want to clip them. We detect outliers 
    # as points in the 99.5 percentile.
    outlier_threshold = 0.005

    for lag in lags:
        df[f"return_{lag}m"] = (df["adj close"]
                                .pct_change(lag)
                                .pipe(lambda x: x.clip(lower = x.quantile(outlier_threshold),upper=x.quantile(1-outlier_threshold)))
                                .add(1)
                                .pow(1/lag)
                                .sub(1))
    
    return df

data = data.groupby(level=1, group_keys=False).apply(calculate_returns).dropna()

data

dollar_volume   adj close  ...  return_9m  return_12m
date       ticker                             ...                       
2017-08-31 A          115.714609   61.394829  ...   0.044742    0.027918
           AAL        250.045330   43.511295  ...  -0.003427    0.018330
           AAPL      4562.601962   38.529819  ...   0.046264    0.038469
           ABBV       308.616295   55.921055  ...   0.027342    0.016730
           ABT        218.410694   45.329674  ...   0.034961    0.018239
...                          ...         ...  ...        ...         ...
2024-07-31 OTIS       183.475646   97.400002  ...   0.027498    0.007045
           ABNB       464.748174  151.960007  ...   0.028222   -0.000126
           CEG        455.455815  215.729996  ...   0.075271    0.069956
           GEHC       152.356552   76.349998  ...   0.015431   -0.001680
           KVUE       253.954599   18.250000  ...   0.001184   -0.018275

[39020 rows x 16 columns]